In [1]:
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
import geopandas as gpd
import copernicusmarine
import datetime

/Users/volkerh/miniforge3/envs/climarest/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def _load_aoi():
    fname = '../data/galicia_mussel_farms.geojson'
    gdf_aoi = gpd.read_file(fname)
    # return gdf_aoi.head(1).geometry.bounds
    return gdf_aoi.head(1).geometry.bounds.values.tolist()[0]

bounds = _load_aoi()
print(bounds)

[-8.95, 42.54, -8.75, 42.68]


In [4]:
def _load_ocean_model(minx, miny, maxx, maxy, mint, maxt):
    # Set parameters
    data_request = {
       # "dataset_id" : "cmems_mod_ibi_phy_anfc_0.027deg-3D_PT1H-m",  # 3D, hourly
       "dataset_id" : "cmems_mod_ibi_phy_anfc_0.027deg-2D_PT1H-m",  # 2D, hourly
       "longitude" : [minx, maxx], 
       "latitude" : [miny, maxy],
       "time" : ["2025-05-10", "2025-05-12"],
       "variables" : ["thetao"]  # sea water potential temperature in C
    }
    
    # Load xarray dataset
    # dset = copernicusmarine.open_dataset(
    #     dataset_id = data_request["dataset_id"],
    #     minimum_longitude = data_request["longitude"][0],
    #     maximum_longitude = data_request["longitude"][1],
    #     minimum_latitude = data_request["latitude"][0],
    #     maximum_latitude = data_request["latitude"][1],
    #     start_datetime = data_request["time"][0],
    #     end_datetime = data_request["time"][1],
    #     variables = data_request["variables"]
    # )

    # Load dataframe
    df = copernicusmarine.read_dataframe(
        dataset_id = data_request["dataset_id"],
        minimum_longitude = data_request["longitude"][0],
        maximum_longitude = data_request["longitude"][1],
        minimum_latitude = data_request["latitude"][0],
        maximum_latitude = data_request["latitude"][1],
        start_datetime = data_request["time"][0],
        end_datetime = data_request["time"][1],
        variables = data_request["variables"]
    )
    
    # Print loaded dataset information
    return df

# set time
tlo = datetime.datetime.now().isoformat()
thi = (datetime.datetime.now() + datetime.timedelta(days=7)).isoformat()

# load
# dset = _load_ocean_model(bounds[0], bounds[1], bounds[2], bounds[3], tlo ,thi)
df = _load_ocean_model(bounds[0], bounds[1], bounds[2], bounds[3], tlo ,thi)

INFO - 2025-05-12T11:50:41Z - Selected dataset version: "202411"
INFO - 2025-05-12T11:50:41Z - Selected dataset part: "default"


In [5]:
# plt.imshow(dset['thetao'][3,:,:])

In [6]:
df_flat = df.reset_index().dropna(subset=['thetao']).reset_index()

In [7]:
df_flat

,index,time,latitude,longitude,thetao
0,1,2025-05-10 00:00:00,42.554749,-8.915863,17.384
1,18,2025-05-10 00:00:00,42.610306,-8.832527,17.461
2,19,2025-05-10 00:00:00,42.610306,-8.804748,17.551
3,36,2025-05-10 01:00:00,42.554749,-8.915863,17.347
4,53,2025-05-10 01:00:00,42.610306,-8.832527,17.420
...,...,...,...,...,...
142,1663,2025-05-11 23:00:00,42.610306,-8.832527,17.455
143,1664,2025-05-11 23:00:00,42.610306,-8.804748,17.499
144,1681,2025-05-12 00:00:00,42.554749,-8.915863,17.304
145,1698,2025-05-12 00:00:00,42.610306,-8.832527,17.419


In [8]:
# df_flat['STATE'] = df_flat[df_flat.thetao>17.6]
df_flat['state'] = df_flat.thetao.apply(lambda thetao: 'ALARM' if thetao>17.6 else 'NOMINAL')

In [9]:
df_flat[df_flat.state=='ALARM']

,index,time,latitude,longitude,thetao,state
125,1454,2025-05-11 17:00:00,42.610306,-8.804748,17.629,ALARM
127,1488,2025-05-11 18:00:00,42.610306,-8.832527,17.613,ALARM
128,1489,2025-05-11 18:00:00,42.610306,-8.804748,17.646,ALARM
130,1523,2025-05-11 19:00:00,42.610306,-8.832527,17.602,ALARM
131,1524,2025-05-11 19:00:00,42.610306,-8.804748,17.637,ALARM
134,1559,2025-05-11 20:00:00,42.610306,-8.804748,17.608,ALARM
